# Viewing JWST Spectra with Spectral DB API and SpecViz

## Learning Goals

By the end of this tutorial, you will:
- Understand the difference between downloading JWST spectra with `astroquery.MAST` and `SpectralDB`
- Access a JWST spectrum with a known URI using `SpectralDB` API
- Search for JWST spectra given a set of criteria. 
- View a JWST spectrum with `SpecViz`

## Introduction

The Barbara A. Mikulski Archive for Space Telescopes (MAST) is the home for all data from the James Webb Space Telescope (JWST), among other telescope data archives. There are multiple ways to search for and download images and spectra from MAST. In this tutorial, we will access a spectrum with a known Uniform Resource Indicator (URI) using the Spectral DB Application Programming Interface (API). Using a correctly formatted URL (Uniform Resource Locator; a web address that starts with https://) request to the API, we can directly access the same content as contained with a .fits file in our Jupyter notebook without having to download and store the .fits file locally. This tutorial will also briefly demonstrate how to plot the spectrum in the `SpecViz` portion of `Jdaviz`. 

At the time of this tutorial, not all publicly available taken by JWST is available through Spectral DB; it currently only contains the final calibrated data science products, not lower level calibration products. Additionally, searching for 2D and 3D spectroscopic pixel data is not yet supported. The filename of a 1D spectroscopic products, which are supported in searching, will all end in `_x1d.fits1`.

JRK, questions, 
- how should this be written consistently... Spectral DB, Spectraldb, etc. `with` or without formatting?
- how do know if the spectrum you are trying to access is final calibrated science data product?
- See how long the "at the time of this tutorial..." will last.

## Imports

- numpy to handle array functions
- astropy.io fits for accessing FITS files
- astropy.io ascii for writing an astropy table to a .csv file (for an Exercise)
- astropy.table Table for creating tidy tables of the data
- matplotlib.pyplot for plotting data
- requests to access the API
- astroquery.mast Observations for querying MAST for observations
- json

JRK go back and see what ends up being used.

In [41]:
%matplotlib inline
import numpy as np
from astropy.io import fits
from astropy.io import ascii
from astropy.table import Table
import matplotlib.pyplot as plt
from astroquery.mast import Observations
import requests
import json

## Using Spectral DB API to access a JWST spectrum with known URI

Each observation in the JWST data archive has a unique URI (Uniform Resource Locator) that consists of a prefix and a file name. For our example, we will use a MIRI spectrum of NGC 6225, a barred spiral galaxy with an active galactic nucleus. 

JRK need to explain or demonstrate some ways to find the URI, presuming you know what you're looking for.

The URLs for the Spectral DB API will all begin with https://mast.stsci.edu/spectra/api/v0.1/retrieve?. In our case, with a known URI, we'll append 'uri=' and then the spectrum's URI. In the code block below, we combine those two strings and print the final result. 

In [26]:
#obs_table=Observations.query_object('NGC 6225')
#obs_table=Observations.query_criteria(objectname='NGC 6225',)

In [27]:
# From 7/18/22:
prefix='https://mast.stsci.edu/spectra/api/v0.1/retrieve?uri='
uri='mast:JWST/product/jw01039-o005_t001_miri_ch3-shortlongmedium-_x1d.fits'
url=prefix+uri
print(url)

https://mast.stsci.edu/spectra/api/v0.1/retrieve?uri=mast:JWST/product/jw01039-o005_t001_miri_ch3-shortlongmedium-_x1d.fits


By using requests.get with our URL, we ask the Spectral DB API to send the spectrum information associated with that unique URI. The resulting object, `r`, is a JSON (JavaScript Object Notation) object, used to store and exchanged data in a standardized format. Next, we print the status code of the `r` object; a code of 200 means success.

In [28]:
r=requests.get(url)
r.status_code # 200 means success.

200

We can view the contents of the JSON object:

In [37]:
r.json()

{'status': 1,
 'message': 'Successfully found data for jw01039-o005_t001_miri_ch3-shortlongmedium-_x1d.fits',
 'uri': 'mast:JWST/product/jw01039-o005_t001_miri_ch3-shortlongmedium-_x1d.fits',
 'filename': 'jw01039-o005_t001_miri_ch3-shortlongmedium-_x1d.fits',
 'column_metadata': {'wavelength': {'dtype': 'float',
   'description': 'wavelength values',
   'units': 'um',
   'fitsExt': 'WAVELENGTH'},
  'flux': {'dtype': 'float',
   'description': 'flux values',
   'units': 'Jy',
   'fitsExt': 'FLUX'},
  'fluxErr': {'dtype': 'float',
   'description': 'error values',
   'units': 'Jy',
   'fitsExt': 'FLUX_ERROR'},
  'fluxVarPoisson': {'dtype': 'float',
   'description': 'error values',
   'units': 'Jy^2',
   'fitsExt': 'FLUX_VAR_POISSON'},
  'fluxVarRnoise': {'dtype': 'float',
   'description': 'error values',
   'units': 'Jy^2',
   'fitsExt': 'FLUX_VAR_RNOISE'},
  'fluxVarFlat': {'dtype': 'float',
   'description': 'error values',
   'units': 'Jy^2',
   'fitsExt': 'FLUX_VAR_FLAT'},
  'sb':

- Explain a JSON object
- How to navigate it

The `r` object itself looks a lot like a dictionary, but to access the data contained inside, we'll need to use the json.load() method.

In [40]:
data=json.load(r)

requests.models.Response

Cool example
Example:  To search for 1d spectroscopic (x1d) JWST data products that have a derived global SNR > 5 and a flux value >= 10 in the wavelength range of 1.2-1.3 microns, the URL route is https://mast.stsci.edu/viz/api/v0.1/search?wavelength=1.2,1.3&flux.gte=10&derSnr.gt=5

https://outerspace.stsci.edu/display/MASTDOCS/API+for+JWST+Science+Pixels
